### Auto Bayesian Neural Networks


autoBNN on EUR/USD

In [3]:
import yfinance as yf
import numpy as np
import pandas as pd
import autobnn as ab
import jax
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

2025-06-27 22:21:57.342257: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751077317.376670  350709 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751077317.390321  350709 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [12]:
eurusd = yf.download("EURUSD=X", start="2005-01-01", end="2025-06-22")
eurusd = eurusd.reset_index()

/tmp/ipykernel_350709/1077654021.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  eurusd = yf.download("EURUSD=X", start="2005-01-01", end="2025-06-22")
[*********************100%***********************]  1 of 1 completed


In [13]:
eurusd['Date'] = pd.to_datetime(eurusd['Date'])

In [14]:
if 'Volume' in eurusd.columns:
    eurusd.drop(columns=['Volume'], inplace=True)

/tmp/ipykernel_350709/607364856.py:2: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  eurusd.drop(columns=['Volume'], inplace=True)


In [15]:
def label_data(df, lookahead=5, threshold=0.002):
    """
    Labels each candle based on future closing price percentage change.
    Parameters:
    -----------
    df : pd.DataFrame
        Data containing at least a 'Close' column.
    lookahead : int, optional
        Number of candles to look ahead (default is 5).
    threshold : float, optional
        Percentage change threshold for classification (default is 0.002 or 0.2%).
    """
    
    # Compute future percentage change in closing price
    df["future_return"] = df["Close"].pct_change(lookahead).shift(-lookahead)
    # Assign labels
    df["label"] = 0  # Default: Neutral
    df.loc[df["future_return"] > threshold, "label"] = 2  # Up
    df.loc[df["future_return"] < -threshold, "label"] = 1  # Down
    # Drop future_return column (not needed in final output)
    df.drop(columns=["future_return"], inplace=True)

In [16]:
label_data(eurusd)

/tmp/ipykernel_350709/783438040.py:21: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df.drop(columns=["future_return"], inplace=True)


In [17]:
print(eurusd[eurusd["label"] != 0].head(10))

Price        Date     Close      High       Low      Open label
Ticker             EURUSD=X  EURUSD=X  EURUSD=X  EURUSD=X      
0      2005-01-03  1.347001  1.358105  1.340195  1.358105     1
1      2005-01-04  1.328198  1.349601  1.326102  1.347001     1
3      2005-01-06  1.318305  1.327898  1.316396  1.327898     2
4      2005-01-07  1.306097  1.324802  1.302999  1.318200     2
5      2005-01-10  1.310994  1.311992  1.306404  1.307600     1
6      2005-01-11  1.311699  1.317107  1.310496  1.311407     1
7      2005-01-12  1.325592  1.329098  1.308695  1.311407     1
8      2005-01-13  1.321109  1.326594  1.320306  1.325908     1
9      2005-01-14  1.309895  1.321196  1.306797  1.321196     1
11     2005-01-18  1.301795  1.307292  1.300306  1.304904     1
